# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(email = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 0,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&obj-id.prefix=10.1101&source=twitter&rows=0&from-occurred-date=2021-06-28&until-occurred-date=2021-07-05


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

14699 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(email = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-28&until-occurred-date=2021-07-05
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=272112ab-32cd-4390-996a-b532662390fe&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-28&until-occurred-date=2021-07-05
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=a40d195b-585b-4d89-b062-a1c7325191d9&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-06-28&until-occurred-date=2021-07-05
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=dd1c9e67-a948-4b81-9bf2-86f4f510ddaf&rows=

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-05-10.csv
failed to load preprint_tweets_2021-05-17.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load preprint_tweets_2021-06-21.csv
failed to load tweets_sorted.json
failed to load preprint_tweets_2021-06-28.csv
failed to load preprint_tweets_2021-06-14.csv
failed to load preprint_tweets_2021-06-07.csv
failed to load preprint_tweets_2021-05-18.csv
failed to load preprint_tweets_2021-05-24.csv
failed to load preprint_tweets_2021-05-31.csv
failed to load preprint_tweets_2021-05-09.csv
failed to load preprint_tweets_2021-05-08.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 5]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []

for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.05.16.21257255...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.06.28.21259673...
REST API query complete  200
REST API query started for 10.1101/2021.03.19.435959...
REST API query complete  200
REST API query started for 10.1101/2021.06.29.450356...
REST API query complete  200
REST API query started for 10.1101/2021.07.01.450676...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.05.21.21257595...
REST API query complete  200
REST API query started for 10.1101/2020.11.15.383323...
REST API query complete  200
REST API query started for 10.1101/2021.06.30.450543...
REST API query complete  200
REST API query started for 10.1101/2021.06.18.449051...
REST API query complete  200
REST API query started for 10.1101/2021.06.30.450524...

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.06.28.21259673,910,medRxiv,Infectious Diseases (except HIV/AIDS),True,Infectivity and immune escape of the new SARS-...,"[{'name': 'Mónica L. Acevedo'}, {'name': 'Luis...",<sec><title>Background</title><p>The newly des...,2021-07-01
1,10.1101/2021.06.29.450356,564,bioRxiv,Immunology,True,Thrombocytopenia and splenic platelet directed...,"[{'name': 'Leo Nicolai'}, {'name': 'Alexander ...",<p>Vaccines against SARS-CoV-2 are based on a ...,2021-06-29
2,10.1101/2021.07.01.450676,513,bioRxiv,Microbiology,True,Neutralization of Delta variant with sera of C...,"[{'name': 'Gajanan Sapkal'}, {'name': 'Pragya ...",<p>The recent emergence of B.1.617 lineage has...,2021-07-02
3,10.1101/2021.06.30.450543,245,bioRxiv,Neuroscience,False,Optogenetic stimulation of medial amygdala GAB...,"[{'name': 'Aiste Baleisyte'}, {'name': 'Ralf S...",<p>Optogenetic manipulation of genetically-spe...,2021-06-30
4,10.1101/2021.06.30.450524,226,bioRxiv,Cell Biology,False,Horizontal Transfer of Histone H3 by Mammalian...,"[{'name': 'Parker L. Sulkowski'}, {'name': 'Hi...","<p>Histones are small, highly basic, nuclear p...",2021-06-30
5,10.1101/2021.06.29.450255,113,bioRxiv,Genomics,False,SLOW5: a new file format enables massive accel...,"[{'name': 'Hasindu Gamaarachchi'}, {'name': 'H...",<p>Nanopore sequencing is an emerging genomic ...,2021-06-30
6,10.1101/2021.06.28.450135,87,bioRxiv,Bioinformatics,False,DeepMP: a deep learning tool to detect DNA bas...,"[{'name': 'Jose Bonet'}, {'name': 'Mandi Chen'...",<sec><title>Motivation</title><p>DNA Methylati...,2021-06-28
7,10.1101/2021.06.29.450332,51,bioRxiv,Neuroscience,False,Cherry-picking information: humans actively sa...,"[{'name': 'Paula Kaanders'}, {'name': 'Pradyum...",<p>No one likes to be wrong. Previous research...,2021-06-30
8,10.1101/2021.07.02.450779,47,bioRxiv,Neuroscience,False,Wild lab: A naturalistic free viewing experime...,"[{'name': 'Anna L. Gert'}, {'name': 'Benedikt ...",<p>Neural mechanisms of face perception are pr...,2021-07-02
9,10.1101/2021.06.28.21259658,41,medRxiv,Infectious Diseases (except HIV/AIDS),True,Post COVID-19 sequelae: A prospective observat...,"[{'name': 'Shivdas Naik'}, {'name': 'Manish So...","<sec><title>Background</title><p>Long COVID, o...",2021-07-01


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 5 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

71 preprints (including 10 covering SARS-CoV-2, 65 from bioRxiv and 6 from medRxiv) published in the last 7 days before 2021-07-05 had been tweeted at least 5 times (maximum 910).